<h1>ASAT </h1>
<style>
    h1{text-align:center}
</style>

<h2 style="text-align:center;margin-bottom:50px">Aeronautics Recruitment: Software Development
Subsystem
</h2>


<div>
    <h3>Autopilot Overview</h3>
    <p>For a UAV to be able to function autonomously, an interactive understanding of its environment is essential. To achieve that, the vehicle needs to integrate a series of sensors that will provide information on its location, orientation, speed, altitude, etc. Such sensors are a 3-axis gyroscope, a GPS (Global Positioning System), an accelerometer, an altimeter, and an airspeed sensor. Due to the objective of project Phoenix (fire detection), a thermal and a regular camera are required. For the UAV to be able to communicate and detect other aircraft, it can use ADS-B technology (Automatic Dependent Surveillance-Broadcast). The hardware mentioned above should comply with the pixhawk open standards to avoid unnecessary incompatibilities</p>
    <p>Apart from the sensor's data, the autopilot needs additional input from the user to plan its route accordingly. The user shall specify the destination, flight altitude, and speed of the aircraft. There should also be an option to add waypoints. With the purpose of fire detection, it should include a mode that allows the user to select an area for scanning, letting the UAV roam above it.</p>
    <p>Autopilot software dfdfd</p>
</div>

<div>
    <h3>Αλγόριθμος του Dijkstra</h3>
    <p></p>
</div>

In [13]:
import numpy as np

In [14]:
connections = [
    [0, 1, 0, 3, 10],
    [1, 0, 5, 0, 0],
    [0, 5, 0, 2, 1],
    [3, 0, 2, 0, 6],
    [10, 0, 1, 6, 0]
    ]

In [15]:
baseNode=0
weight=[]
unvisited=[]
minimum=0

In [16]:
for i in range(0, len(connections)):
    weight.append(np.infty)
    unvisited.append(i)
weight[baseNode]=0

In [17]:
while len(unvisited) != 0:
    for i in unvisited:
        if minimum>=weight[i]:
            minimum=weight[i]
    i=weight.index(minimum)
    unvisited.remove(i)
    for j in range(0, len(connections)):
        if connections[i][j]!=0:
            weight[j]=min(weight[j], weight[i]+connections[i][j])
    minimum=max(weight)

In [18]:
print("==========================================")
print("Starting from node " + str(baseNode+1) + " :")
print("Minimum distance to other nodes:\n")
print("Node    Distance\n---------------")

for j in range(0, len(connections)):
    print(j+1, "\t\t", weight[j])

print("\n==========================================")

Starting from node 1 :
Minimum distance to other nodes:

Node    Distance
---------------
1 		 0
2 		 1
3 		 5
4 		 3
5 		 6



<div>
    <h3>Κατηγοριοποιήση ψηφίων</h3>
    <p></p>
</div>